# Imports

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tabulate as tabulate
from matplotlib.ticker import FuncFormatter
from mpl_toolkits.mplot3d import Axes3D
import plotly.express as px
from matplotlib.colors import TwoSlopeNorm

pd.set_option('display.float_format', '{:,.0f}'.format)

# From previous work

## Importing Electricity

In [2]:
elecdf=pd.read_csv(r"C:\Users\tonya\OneDrive - Rutgers University\Documents\Rutgers\Research\NRELAnalysis\Electricity.csv")

elecMAWNC=elecdf[elecdf['Division'].isin(['Mid Atlantic', 'West North Central'])]
elecMAWNC=elecMAWNC.drop('Used on Plant (Cent/s)', axis=1)
elecMAWNC=elecMAWNC.drop('Total Electriciy (Cent/s)', axis=1)
elecMAWNC['Price ($/kWh)']=elecMAWNC['Industrial 2024 (Cent/kWh)']/100

In [21]:
FAthermo={
    'Compound': ['CO2', 'H2O', 'O2', 'CH2O2'], 
    'H form': [-393.5, -285.8, 0, -425], 
    'G form': [-394.4, -237.1, 0, -361.4], 
}
FAtherm_df=pd.DataFrame(FAthermo)
FA=FAtherm_df.loc[FAtherm_df['Compound']=='CH2O2']
CO2=FAtherm_df.loc[FAtherm_df['Compound']=='CO2']
H2O=FAtherm_df.loc[FAtherm_df['Compound']=='H2O']
H_rxn=FA['H form'].values[0] - (CO2['H form'].values[0] + H2O['H form'].values[0])
G_rxn=FA['G form'].values[0] - (CO2['G form'].values[0] + H2O['G form'].values[0])
print(f"H rxn: {H_rxn:.1f}")
print(f"G rxn: {G_rxn:.1f}")

genelec=12797
FAprod=genelec/G_rxn
print(f"Formic Acid Produced: {FAprod:.1f} mol/s")
FAmass=(FAprod*46.03)/1000 #kg/s

H rxn: 254.3
G rxn: 270.1
Formic Acid Produced: 47.4 mol/s


## Reference Ethanol Electrolyzer from Jouny Paper

In [22]:
refEV=1.75 #Reference electrolyzer voltage in V
refCD=0.175 #Reference electrolyzer current density in A/cm^2
refprod=100000 #kg/day of ethanol

#Electrolyzer Assumptions
FarEff=0.9 #90% Faradaic efficiency
elpm=12 #12 electrons per mole of ethanol
MWFA=46.06 #molecular weight of FA in g/mol
fc=96480 #Faradays Constant in C/mol
kgtog=1000 #conversion of kg to g
totcur=((refprod/86400)*kgtog*(1/MWFA)*elpm*fc)/FarEff
elecareacalc=(totcur/refCD)*(1/(10**4))
print(f"Electrolyzer Area: {elecareacalc:.1f} m^2")
powerneed=(totcur*refEV)/(10**6)
print(f"Total Power Required: {powerneed:.1f} MW")
elecreq=0.12*powerneed*1000*8000
print(f"Price of Electricity Required for Electrolyzer in NJ: ${elecreq:,.0f}/yr")

ezrcost={
    'Base Cost ($/kW)': [250, 550, 2640], 
}
ezrcost_df=pd.DataFrame(ezrcost)
ezrcost_df['Installed Cost ($/m^2)']=ezrcost_df['Base Cost ($/kW)']*refEV*refCD/1000*(10**4)*1.2
ezrcost_df['Electrolyzer Capital Cost ($)']=ezrcost_df['Installed Cost ($/m^2)']*elecareacalc

annFA=refprod*(8000/24)/1000 #kg/day * 1day/24h* 8000h/yr *1ton/1000 kg
sellannFA=annFA*520
print(f"Annual Profit of Ethanol: ${sellannFA:,.0f}/yr")
annCO2=annFA*(44.01/46.06) #tonFA/yr*1000kg/1ton*1kmolFA/46.03kg*1kmolCO2/1kmolFA*44.01kg/1kmolCO2*1ton/1000kg
sellannCO2=annCO2*40
print(f"Annual Price of CO2 at $40/ton: ${sellannCO2:,.0f}/yr")
yearprof=sellannFA-elecreq-sellannCO2
print(f"Annual Profit of Electrolyzer: ${yearprof:,.0f}/yr")

ezrcost_df['Payback Period (years)']=(ezrcost_df['Electrolyzer Capital Cost ($)']/yearprof).apply(lambda x: f"{x:,.2f}")
print(ezrcost_df.to_string(index=False, justify='center'))

Electrolyzer Area: 18471.4 m^2
Total Power Required: 56.6 MW
Price of Electricity Required for Electrolyzer in NJ: $54,305,978/yr
Annual Profit of Ethanol: $17,333,333/yr
Annual Price of CO2 at $40/ton: $1,273,990/yr
Annual Profit of Electrolyzer: $-38,246,635/yr
 Base Cost ($/kW)  Installed Cost ($/m^2)  Electrolyzer Capital Cost ($) Payback Period (years)
        250                  919                    16,970,618                   -0.44         
        550                2,021                    37,335,360                   -0.98         
       2640                9,702                   179,209,726                   -4.69         


## Using parameters from Jouny Paper for Formic Acid

In [23]:
#Electrolyzer Assumptions
cellvolt=2 #in V
prodsel=0.90 #85% product selectiviy, assuming this is where purity of product comes into play. Also is the Faradaic efficiency
elpm=2 #2 electrons per mole of Formic Acid
MWFA=46.03 #molecular weight of FA in g/mol
fc=96480 #Faradays Constant in C/mol
kgtog=1000 #conversion of kg to g
curd=.3 #current density in A/cm^2
totcur=((FAmass*kgtog)/MWFA)*elpm*fc/prodsel
elecareacalc=(totcur/curd)*(1/(10**4))
print(f"Electrolyzer Area: {elecareacalc:.1f} m^2")
powerneed=(totcur*cellvolt)/(10**6)
print(f"Total Power Required: {powerneed:.1f} MW")
elecreq=0.12*powerneed*1000*8000
print(f"Price of Electricity Required for Electrolyzer in NJ: ${elecreq:,.0f}/yr")

ezrcost={
    'Base Cost ($/kW)': [250, 550, 2640], 
}
ezrcost_df=pd.DataFrame(ezrcost)
ezrcost_df['Installed Cost ($/m^2)']=ezrcost_df['Base Cost ($/kW)']*cellvolt*curd/1000*(10**4)*1.2
ezrcost_df['Electrolyzer Capital Cost ($)']=ezrcost_df['Installed Cost ($/m^2)']*elecareacalc

annFA=FAmass*(60*60*8000)/1000 #kg/s * 60s/min*60min/h* 8000h/yr *1ton/1000 kg
sellannFA=annFA*550
print(f"Annual Profit of Formic Acid: ${sellannFA:,.0f}/yr")
annCO2=annFA*(44.01/46.03) #tonFA/yr*1000kg/1ton*1kmolFA/46.03kg*1kmolCO2/1kmolFA*44.01kg/1kmolCO2*1ton/1000kg
sellannCO2=annCO2*40
print(f"Annual Price of CO2 at $40/ton: ${sellannCO2:,.0f}/yr")
yearprof=sellannFA-elecreq-sellannCO2
print(f"Annual Profit of Electrolyzer: ${yearprof:,.0f}/yr")

ezrcost_df['Payback Period (years)']=(ezrcost_df['Electrolyzer Capital Cost ($)']/yearprof).apply(lambda x: f"{x:,.2f}")
print(ezrcost_df.to_string(index=False, justify='center'))

Electrolyzer Area: 3386.0 m^2
Total Power Required: 20.3 MW
Price of Electricity Required for Electrolyzer in NJ: $19,503,367/yr
Annual Profit of Formic Acid: $34,544,566/yr
Annual Price of CO2 at $40/ton: $2,402,080/yr
Annual Profit of Electrolyzer: $12,639,119/yr
 Base Cost ($/kW)  Installed Cost ($/m^2)  Electrolyzer Capital Cost ($) Payback Period (years)
        250                 1,800                    6,094,802                    0.48         
        550                 3,960                   13,408,565                    1.06         
       2640                19,008                   64,361,111                    5.09         


## Using Summary Table for Formic Acid from Huang paper

In [31]:
#Electrolyzer Assumptions
cellvolt=3.5 #in V
prodsel=0.94 #85% product selectiviy, assuming this is where purity of product comes into play. Also is the Faradaic efficiency
elpm=2 #2 electrons per mole of Formic Acid
MWFA=46.03 #molecular weight of FA in g/mol
fc=96480 #Faradays Constant in C/mol
kgtog=1000 #conversion of kg to g
curd=.14 #current density in A/cm^2
totcur=((FAmass*kgtog)/MWFA)*elpm*fc/prodsel
elecareacalc=(totcur/curd)*(1/(10**4))
print(f"Electrolyzer Area: {elecareacalc:.1f} m^2")
powerneed=(totcur*cellvolt)/(10**6)
print(f"Total Power Required: {powerneed:.1f} MW")
elecreq=0.12*powerneed*1000*8000
print(f"Price of Electricity Required for Electrolyzer in NJ: ${elecreq:,.0f}/yr")

ezrcost={
    'Base Cost ($/kW)': [250, 550, 2640], 
}
ezrcost_df=pd.DataFrame(ezrcost)
ezrcost_df['Installed Cost ($/m^2)']=ezrcost_df['Base Cost ($/kW)']*cellvolt*curd/1000*(10**4)*1.12
ezrcost_df['Electrolyzer Capital Cost ($)']=ezrcost_df['Installed Cost ($/m^2)']*elecareacalc

annFA=FAmass*(60*60*8000)/1000 #kg/s * 60s/min*60min/h* 8000h/yr *1ton/1000 kg
sellannFA=annFA*550
print(f"Annual Profit of Formic Acid: ${sellannFA:,.0f}/yr")
annCO2=annFA*(44.01/46.03) #tonFA/yr*1000kg/1ton*1kmolFA/46.03kg*1kmolCO2/1kmolFA*44.01kg/1kmolCO2*1ton/1000kg
sellannCO2=annCO2*40
print(f"Annual Price of CO2 at $40/ton: ${sellannCO2:,.0f}/yr")
yearprof=sellannFA-elecreq-sellannCO2
print(f"Annual Profit of Electrolyzer: ${yearprof:,.0f}/yr")

ezrcost_df['Payback Period (years)']=(ezrcost_df['Electrolyzer Capital Cost ($)']/yearprof).apply(lambda x: f"{x:,.2f}")
print(ezrcost_df.to_string(index=False, justify='center'))

Electrolyzer Area: 6947.0 m^2
Total Power Required: 34.0 MW
Price of Electricity Required for Electrolyzer in NJ: $32,678,514/yr
Annual Profit of Formic Acid: $34,544,566/yr
Annual Price of CO2 at $40/ton: $2,402,080/yr
Annual Profit of Electrolyzer: $-536,028/yr
 Base Cost ($/kW)  Installed Cost ($/m^2)  Electrolyzer Capital Cost ($) Payback Period (years)
        250                 1,372                    9,531,233                   -17.78        
        550                 3,018                   20,968,713                   -39.12        
       2640                14,488                  100,649,823                  -187.77        


## Using Supp Info from Huang

In [33]:
#Electrolyzer Assumptions
cellvolt=1.9 #in V
prodsel=0.94 #85% product selectiviy, assuming this is where purity of product comes into play. Also is the Faradaic efficiency
elpm=2 #2 electrons per mole of Formic Acid
MWFA=46.03 #molecular weight of FA in g/mol
fc=96480 #Faradays Constant in C/mol
kgtog=1000 #conversion of kg to g
curd=2 #current density in A/cm^2
totcur=((FAmass*kgtog)/MWFA)*elpm*fc/prodsel
elecareacalc=(totcur/curd)*(1/(10**4))
print(f"Electrolyzer Area: {elecareacalc:.1f} m^2")
powerneed=(totcur*cellvolt)/(10**6)
print(f"Total Power Required: {powerneed:.1f} MW")
elecreq=0.12*powerneed*1000*8000
print(f"Price of Electricity Required for Electrolyzer in NJ: ${elecreq:,.0f}/yr")

ezrcost={
    'Base Cost ($/kW)': [250, 550, 2640], 
}
ezrcost_df=pd.DataFrame(ezrcost)
ezrcost_df['Installed Cost ($/m^2)']=ezrcost_df['Base Cost ($/kW)']*cellvolt*curd/1000*(10**4)*1.12
ezrcost_df['Electrolyzer Capital Cost ($)']=ezrcost_df['Installed Cost ($/m^2)']*elecareacalc

annFA=FAmass*(60*60*8000)/1000 #kg/s * 60s/min*60min/h* 8000h/yr *1ton/1000 kg
sellannFA=annFA*550
print(f"Annual Profit of Formic Acid: ${sellannFA:,.0f}/yr")
annCO2=annFA*(44.01/46.03) #tonFA/yr*1000kg/1ton*1kmolFA/46.03kg*1kmolCO2/1kmolFA*44.01kg/1kmolCO2*1ton/1000kg
sellannCO2=annCO2*40
print(f"Annual Price of CO2 at $40/ton: ${sellannCO2:,.0f}/yr")
yearprof=sellannFA-elecreq-sellannCO2
print(f"Annual Profit of Electrolyzer: ${yearprof:,.0f}/yr")

ezrcost_df['Payback Period (years)']=(ezrcost_df['Electrolyzer Capital Cost ($)']/yearprof).apply(lambda x: f"{x:,.2f}")
print(ezrcost_df.to_string(index=False, justify='center'))

Electrolyzer Area: 486.3 m^2
Total Power Required: 18.5 MW
Price of Electricity Required for Electrolyzer in NJ: $17,739,765/yr
Annual Profit of Formic Acid: $34,544,566/yr
Annual Price of CO2 at $40/ton: $2,402,080/yr
Annual Profit of Electrolyzer: $14,402,721/yr
 Base Cost ($/kW)  Installed Cost ($/m^2)  Electrolyzer Capital Cost ($) Payback Period (years)
        250                10,640                    5,174,098                    0.36         
        550                23,408                   11,383,016                    0.79         
       2640               112,358                   54,638,475                    3.79         
